In [5]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from sklearn.ensemble import RandomForestClassifier

In [6]:
SAMPLE_RATE = 22050
labels = np.array(['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music'])

### Load Data

In [73]:
# chunksize = 500
# n_chunks = 0

# X_train = np.empty((0, 193))
# y_train = np.empty((0, 1), dtype=int)

In [61]:
# mfcc0 40, chroma 12, mel 7, contrast 128, tonnetz 6
for df in pd.read_csv('./train.csv', header=None, chunksize=chunksize):
    n_chunks += 1
    print 'Chunk #', n_chunks
    
    X_train0 = df.values[:, :-1]
    y_train0 = df.values[:, -1].reshape(-1, 1).astype(int)
    
    mfcc0 = np.array([np.mean(librosa.feature.mfcc(y=x, sr=SAMPLE_RATE, n_mfcc=40).T, axis=0) for x in X_train0])
    chroma = np.array([np.mean(librosa.feature.chroma_stft(S=np.abs(librosa.stft(x)), sr=SAMPLE_RATE).T, axis=0) for x in X_train0])
    contrast = np.array([np.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(x)), sr=SAMPLE_RATE).T, axis=0) for x in X_train0])
    mel = np.array([np.mean(librosa.feature.melspectrogram(x, sr=SAMPLE_RATE).T, axis=0) for x in X_train0])
    tonnetz = np.array([np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(x), sr=SAMPLE_RATE).T, axis=0) for x in X_train0])
    feature0 = np.hstack([mfcc0, chroma, mel, contrast, tonnetz])
    print mfcc0.shape, chroma.shape, contrast.shape, mel.shape, tonnetz.shape, feature0.shape
    
    X_train = np.vstack([X_train, feature0])
    y_train =  y_train0 if n_chunks == 1 else np.vstack([y_train, y_train0])
    print X_train.shape[0], 'samples'
    if n_chunks > 1:
        break
    
print 'X_train_mfcc.shape', X_train.shape, 'y_train.shape', y_train.shape
N_train = X_train.shape[0]

Chunk # 1
(500L, 40L) (500L, 12L) (500L, 7L) (500L, 128L) (500L, 6L) (500L, 193L)
500 samples
Chunk # 2
(500L, 40L) (500L, 12L) (500L, 7L) (500L, 128L) (500L, 6L) (500L, 193L)
1000 samples
X_train_mfcc.shape (1000L, 193L) y_train.shape (1000L, 1L)


In [62]:
X_train_mfcc = X_train[:, :40]
X_train_chroma = X_train[:, 40: 52]
X_train_mel = X_train[:, 52: 59]
X_train_contrast = X_train[:, 59: 187]
X_train_tonnetz = X_train[:, 187:]

### Random Forest

In [85]:
MAX_DEPTH = 10
N_ESTIMATORS = 100

In [86]:
RF = RandomForestClassifier(class_weight='balanced', n_jobs=-1, max_depth=MAX_DEPTH, n_estimators=N_ESTIMATORS)
RF.fit(X_train, y_train)

c:\python27\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=10, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [87]:
RF.score(X_train, y_train)

1.0

In [71]:
RF.predict(X_train)

array([0, 8, 4, 5, 0, 4, 9, 9, 0, 9, 2, 9, 0, 8, 5, 0, 9, 7, 2, 8, 0, 7,
       9, 5, 9, 0, 7, 0, 2, 9, 8, 9, 5, 0, 2, 5, 8, 9, 2, 9, 7, 7, 9, 0,
       3, 0, 7, 7, 1, 0, 5, 0, 2, 9, 2, 8, 7, 0, 7, 2, 5, 8, 0, 4, 9, 2,
       4, 4, 0, 7, 9, 0, 5, 9, 2, 4, 0, 0, 4, 0, 7, 7, 0, 7, 9, 0, 0, 2,
       9, 8, 4, 7, 5, 7, 5, 9, 8, 8, 9, 0, 7, 7, 0, 1, 7, 4, 0, 0, 9, 7,
       7, 7, 4, 7, 0, 8, 9, 0, 3, 2, 0, 9, 7, 9, 7, 9, 0, 2, 9, 5, 3, 7,
       0, 5, 0, 1, 7, 4, 9, 5, 3, 0, 9, 3, 1, 9, 9, 7, 8, 8, 7, 4, 8, 0,
       5, 8, 9, 2, 2, 4, 0, 5, 0, 8, 3, 8, 0, 9, 0, 4, 8, 0, 9, 0, 5, 4,
       2, 3, 2, 5, 0, 2, 8, 2, 7, 7, 1, 0, 2, 2, 8, 9, 8, 7, 8, 8, 8, 4,
       7, 9, 8, 3, 2, 8, 0, 9, 7, 0, 2, 5, 8, 3, 8, 5, 0, 2, 3, 0, 3, 2,
       0, 3, 2, 5, 8, 0, 5, 4, 3, 8, 3, 9, 7, 8, 2, 9, 0, 0, 9, 5, 7, 9,
       4, 0, 4, 2, 2, 2, 3, 0, 0, 0, 3, 2, 4, 7, 7, 7, 2, 8, 2, 7, 4, 9,
       9, 4, 3, 0, 0, 2, 7, 7, 9, 8, 3, 9, 5, 0, 9, 5, 4, 9, 2, 3, 2, 4,
       9, 8, 9, 9, 1, 8, 7, 0, 7, 0, 4, 9, 9, 7, 4,

### Make Prediction

In [139]:
chunksize = 200
n_chunks = 0
outputfile = open('x_test.csv.npy', 'a')

X_test = np.empty((0, 233))
# mfcc0 40, mfcc_std0 40, chroma 12, mel 7, contrast 128, tonnetz 6
for df in pd.read_csv('./test.csv', header=None, chunksize=chunksize):
    n_chunks += 1
    print 'Chunk #', n_chunks
    
    X_test0 = df.values[:, 1:]
    print X_test0.shape
    
    mfcc0 = np.array([np.mean(librosa.feature.mfcc(y=x, sr=SAMPLE_RATE, n_mfcc=40).T, axis=0) for x in X_test0])
    mfcc_std0 = np.array([np.std(librosa.feature.mfcc(y=x, sr=SAMPLE_RATE, n_mfcc=40).T, axis=0) for x in X_test0])
    chroma = np.array([np.mean(librosa.feature.chroma_stft(S=np.abs(librosa.stft(x)), sr=SAMPLE_RATE).T, axis=0) for x in X_test0])
    contrast = np.array([np.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(x)), sr=SAMPLE_RATE).T, axis=0) for x in X_test0])
    mel = np.array([np.mean(librosa.feature.melspectrogram(x, sr=SAMPLE_RATE).T, axis=0) for x in X_test0])
    tonnetz = np.array([np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(x), sr=SAMPLE_RATE).T, axis=0) for x in X_test0])
    feature0 = np.hstack([mfcc0, mfcc_std0, chroma, mel, contrast, tonnetz])
    print mfcc0.shape, chroma.shape, contrast.shape, mel.shape, tonnetz.shape, feature0.shape
    
    X_test = np.vstack([X_test, feature0])
    print X_test.shape[0], 'samples'
    
np.savetxt(outputfile, X_test)
outputfile.close()

Chunk # 1
(200L, 88200L)
(200L, 40L) (200L, 12L) (200L, 7L) (200L, 128L) (200L, 6L) (200L, 233L)
200 samples
Chunk # 2
(200L, 88200L)
(200L, 40L) (200L, 12L) (200L, 7L) (200L, 128L) (200L, 6L) (200L, 233L)
400 samples
Chunk # 3
(200L, 88200L)
(200L, 40L) (200L, 12L) (200L, 7L) (200L, 128L) (200L, 6L) (200L, 233L)
600 samples
Chunk # 4
(200L, 88200L)
(200L, 40L) (200L, 12L) (200L, 7L) (200L, 128L) (200L, 6L) (200L, 233L)
800 samples
Chunk # 5
(200L, 88200L)
(200L, 40L) (200L, 12L) (200L, 7L) (200L, 128L) (200L, 6L) (200L, 233L)
1000 samples


In [149]:
tmp_test.shape

(1000L, 233L)

In [145]:
outputfile.close()

In [146]:

np.save('x_test.csv.npy', X_test)

In [144]:
X_test.shape

(1000L, 233L)

In [88]:
y_hat = RF.predict(X_test)

ValueError: Found array with 0 sample(s) (shape=(0, 193)) while a minimum of 1 is required.

In [82]:
def write_to_file(filename, predictions):
    with open(filename, 'w') as f:
        for i, p in enumerate(predictions):
            f.write(str(i + 1) + ',' + str(p) + '\n')

In [83]:
result = np.vstack((range(len(preds)), preds)).T
df = pd.DataFrame(result)
df = df.astype(int)
df.to_csv('sample_result.csv', index=False, header=['Id', 'Prediction'])